---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
def graph_identification():
    
    # Your Code Here
    out = []
    for graph in P1_Graphs:
        cls_coeff = nx.average_clustering(graph)
        degree = graph.degree()
        degree_values = set(degree.values())
        if len(degree_values) > 10:
            out.append('PA')
        elif cls_coeff < 0.1:
            out.append('SW_H')
        else:
            out.append('SW_L')
    return out # Your Answer Here

In [5]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [6]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [7]:
def salary_predictions():
    from sklearn.metrics import roc_auc_score
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler

    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df_pred = df[df['ManagementSalary'].isnull()]
    df_train = df[df['ManagementSalary'].notnull()]
    X_train, X_test, y_train, y_test = train_test_split(df_train[['Department','clustering','closeness','betweeness']], df_train['ManagementSalary'],random_state=0)
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    model = RandomForestRegressor(random_state=0)
    param_grid = {'n_estimators':[500], 'max_depth':[100]}
    grid_search = GridSearchCV(model, param_grid, scoring="roc_auc")
    grid_result = model.fit(X_train_scaled, y_train)
    answer = grid_result.predict(X_test_scaled)
    #print(roc_auc_score(y_test, answer))
    scaled_input = scaler.transform(df_pred[['Department','clustering','closeness','betweeness']])
    return pd.Series(grid_result.predict(scaled_input),index=df_pred.index)


In [8]:
salary_predictions()

1       0.0
2       1.0
5       1.0
8       0.2
14      0.0
18      0.0
27      0.0
30      1.0
31      0.1
34      0.0
37      0.0
40      0.1
45      0.0
54      0.3
55      0.9
60      0.0
62      1.0
65      1.0
77      0.0
79      0.0
97      0.0
101     0.0
103     0.6
108     0.0
113     0.0
122     0.0
141     0.3
142     1.0
144     0.0
145     0.4
       ... 
913     0.0
914     0.0
915     0.0
918     0.1
923     0.0
926     0.0
931     0.0
934     0.0
939     0.0
944     0.0
945     0.0
947     0.1
950     0.0
951     0.0
953     0.0
959     0.0
962     0.0
963     0.2
968     0.0
969     0.0
974     0.0
984     0.0
987     0.0
989     0.0
991     0.0
992     0.0
994     0.0
996     0.0
1000    0.0
1001    0.2
Length: 252, dtype: float64

In [9]:
#df['degree'] = pd.Series(G.degree())
#df['pr'] = pd.Series(nx.pagerank(G))
#df['degree_centrality'] = pd.Series(nx.degree_centrality(G))

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [66]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [70]:
def new_connections_predictions():
    from sklearn.metrics import roc_auc_score
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import train_test_split
    
    f_c = future_connections
    f_c.reset_index(inplace=True)
    f_c['jacard_coeff'] = [x[2] for x in nx.jaccard_coefficient(G,f_c['index'])]
    f_pred = f_c[f_c['Future Connection'].isnull()]
    f_t = f_c[f_c['Future Connection'].notnull()]
    X = f_t['jacard_coeff']
    y = f_t['Future Connection']
    X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)
    model = RandomForestRegressor(random_state=0)
    param_grid = {'n_estimators':[500], 'max_depth':[100]}
    grid_search = GridSearchCV(model, param_grid, scoring="roc_auc")
    grid_result = model.fit(X_train.reshape(-1, 1), y_train)
    pred = f_pred['jacard_coeff']
    index = f_pred['index']
    out = pd.Series(grid_result.predict(pred.reshape(-1, 1)),index=index)
    
    return out# Your Answer Here

In [71]:
new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


index
(107, 348)    0.032922
(542, 751)    0.012724
(20, 426)     0.366871
(50, 989)     0.012724
(942, 986)    0.012724
(324, 857)    0.012724
(13, 710)     0.029843
(19, 271)     0.063814
(319, 878)    0.012724
(659, 707)    0.012724
(49, 843)     0.012724
(208, 893)    0.012724
(377, 469)    0.012724
(405, 999)    0.000000
(129, 740)    0.079690
(292, 618)    0.024087
(239, 689)    0.012724
(359, 373)    0.012724
(53, 523)     0.033737
(276, 984)    0.012724
(202, 997)    0.012724
(604, 619)    0.578893
(270, 911)    0.012724
(261, 481)    0.070594
(200, 450)    0.729448
(213, 634)    0.012724
(644, 735)    0.437401
(346, 553)    0.012724
(521, 738)    0.012724
(422, 953)    0.026255
                ...   
(672, 848)    0.012724
(28, 127)     0.718361
(202, 661)    0.012724
(54, 195)     1.000000
(295, 864)    0.012724
(814, 936)    0.012724
(839, 874)    0.012724
(139, 843)    0.012724
(461, 544)    0.012724
(68, 487)     0.012724
(622, 932)    0.012724
(504, 936)    0.001565
(479,